In [25]:
import joblib
import function_library as fl
import importlib
import pandas as pd
importlib.reload(fl)

<module 'function_library' from 'C:\\Users\\leere\\PycharmProjects\\Football_ML3\\function_library.py'>

In [26]:
file_path = r"C:\Users\leere\OneDrive\Desktop\RAW DATA\GOALS_LIVE.CSV"

# List of common encodings to try
encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']

data = None
for encoding in encodings:
    try:
        data = pd.read_csv(file_path, encoding=encoding)
        print(f"Successfully read the file with encoding: {encoding}")
        break
    except UnicodeDecodeError:
        print(f"Failed to decode with encoding: {encoding}")

col_dict = {
    "country": "Country",
    "league": "League",
    "datameci": "Date",
    "etapa": "Round",
    "txtechipa1": "home_team",
    "txtechipa2": "away_team",
    "place1t": "Home_team_place_total",
    "place1a": "Home_team_place_home",
    "place2t": "Away_team_place_total",
    "place2d": "Away_team_place_away",
    "customh": "ELO_home",
    "customa": "ELO_away",
    "custom3": "FORM_home",
    "custom4": "FORM_away",
    "home_val": "home_win",
    "home_val_2": "home_win_15",
    "home_val_3": "home_o25",
    "home_val_4": "home_o35",
    "home_val_5": "home_scored",
    "away_val": "away_win",
    "away_val_2": "away_win_15",
    "away_val_3": "away_o25",
    "away_val_4": "away_o35",
    "away_val_5": "away_scored",
    "scor1": "home_goals",
    "scor2": "away_goals",
    "cotao": "o2.5_odds",
}

data = data.rename(columns=col_dict).filter(items=col_dict.values())
# Convert Date column to datetime type
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

# Order by date
data = data.sort_values(by='Date')

data = data[data["Round"] >= 8]
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

cols_to_drop = ['Date','home_team', 'away_team', 'home_goals', 'away_goals', 'o2.5_odds']

data_ready =data.drop(columns=cols_to_drop)
data_ready

Successfully read the file with encoding: utf-8


,Country,League,Round,Home_team_place_total,Home_team_place_home,Away_team_place_total,Away_team_place_away,ELO_home,ELO_away,FORM_home,...,home_win,home_win_15,home_o25,home_o35,home_scored,away_win,away_win_15,away_o25,away_o35,away_scored
0,Scotland,Premier L,11,8,10,2,3,1785.0,2095.0,-15.0,...,40.000,30.000,100.000,60.000,2.100,30.000,10.000,60.000,20.000,1.200
1,Netherlands,Eerste Divisie,21,15,7,13,18,1314.0,1443.0,2.0,...,50.000,10.000,60.000,30.000,1.200,10.000,0.000,40.000,40.000,1.100
2,Netherlands,Eerste Divisie,21,10,6,5,1,1507.0,1473.0,-12.0,...,55.556,22.222,44.444,11.111,1.333,70.000,30.000,40.000,40.000,2.000
3,France,Ligue 2,18,6,9,13,9,1609.0,1466.0,20.0,...,50.000,25.000,50.000,37.500,1.625,33.333,11.111,44.444,22.222,1.111
4,Netherlands,Eerste Divisie,21,4,8,20,19,1481.0,1196.0,5.0,...,40.000,30.000,50.000,40.000,2.000,0.000,0.000,44.444,33.333,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Netherlands,Eredivisie,18,10,10,11,16,1629.0,1716.0,11.0,...,62.500,12.500,62.500,12.500,1.375,11.111,0.000,77.778,11.111,0.444
140,Netherlands,Eredivisie,18,14,11,17,17,1699.0,1598.0,1.0,...,37.500,25.000,25.000,25.000,1.125,0.000,0.000,50.000,12.500,0.250
141,Netherlands,Eredivisie,18,4,6,3,2,2120.0,1912.0,-6.0,...,62.500,37.500,50.000,25.000,2.125,75.000,25.000,75.000,37.500,2.125
142,Italy,Serie B,21,20,20,7,10,1408.0,1573.0,4.0,...,20.000,0.000,80.000,40.000,1.200,11.111,0.000,33.333,22.222,1.000


In [27]:
# Directory where models are saved
models_dir = "TOP_5_MODELS"

# Group unseen data by League and Sub-League
grouped_data = data_ready.groupby(['Country', 'League'])

# Dictionary to store predictions
predictions = {}

# Iterate through each group in the unseen data
for (country, league), group in grouped_data:
    # Construct the model filename
    model_filename = f"{models_dir}/{country}_{league}_model.pkl"

    try:
        # Load the corresponding model
        saved_data = joblib.load(model_filename)  # Load the entire dictionary
        model = saved_data['model']  # Extract the model
        scaler = saved_data['scaler']  # Extract the scaler
        threshold = saved_data['threshold']  # Extract the threshold

        # Extract features for prediction (exclude grouping columns)
        X = group.drop(columns=['Country', 'League'])

        # Scale the features
        X_scaled = scaler.transform(X)

        # Make probability predictions using the model
        y_proba = model.predict_proba(X_scaled)[:, 1]  # Positive class probabilities

        # Filter predictions based on the threshold
        group['Probability'] = y_proba
        group['Prediction'] = (y_proba >= threshold).astype(int)

        # Keep only rows where Prediction == 1
        filtered_predictions = group[group['Prediction'] == 1]

        # Add `home_team` and `away_team` from the original `data`
        game_info = data.loc[filtered_predictions.index, ['home_team', 'away_team', 'Date']]
        filtered_predictions = pd.concat([game_info, filtered_predictions], axis=1)

        # Add filtered predictions to the dictionary
        predictions[(country, league)] = filtered_predictions
        print(f"Filtered predictions made for {country} - {league}")

    except FileNotFoundError:
        print(f"Model not found for {country} - {league}. Skipping...")

    except Exception as e:
        print(f"Error processing {country} - {league}: {e}")

# Combine all predictions into a single DataFrame
if predictions:
    all_predictions = pd.concat(predictions.values(), ignore_index=True)
    print("\nAll Predictions as a DataFrame:")
    print(all_predictions)
else:
    print("No predictions available.")


Model not found for Australia - A-League. Skipping...
Model not found for Belgium - Pro League. Skipping...
Model not found for England - League One. Skipping...
Model not found for England - League Two. Skipping...
Filtered predictions made for France - Ligue 1
Model not found for France - Ligue 2. Skipping...
Filtered predictions made for Germany - Bundesliga
Model not found for Greece - Super L. Skipping...
Model not found for Israel - Premier League. Skipping...
Filtered predictions made for Italy - Serie A
Model not found for Italy - Serie B. Skipping...
Model not found for Netherlands - Eerste Divisie. Skipping...
Model not found for Netherlands - Eredivisie. Skipping...
Model not found for Scotland - Championship. Skipping...
Model not found for Scotland - Premier L. Skipping...
Filtered predictions made for Spain - Primera
Model not found for Spain - Segunda. Skipping...
Model not found for Turkey - 1. Lig. Skipping...
Model not found for Turkey - Super Lig. Skipping...

All Pr

In [28]:
#all_predictions

In [29]:
all_predictions = fl.team_name_map(all_predictions)
all_predictions

,home_team,away_team,Date,Country,League,Round,Home_team_place_total,Home_team_place_home,Away_team_place_total,Away_team_place_away,...,home_o25,home_o35,home_scored,away_win,away_win_15,away_o25,away_o35,away_scored,Probability,Prediction
0,Auxerre,Lille,2025-01-10,France,Ligue 1,17,9,3,4,4,...,75.000,25.000,1.750,37.500,37.500,37.500,25.000,1.500,0.931788,1
1,Toulouse,Strasbourg,2025-01-12,France,Ligue 1,17,8,11,11,13,...,37.500,12.500,1.250,12.500,12.500,75.000,50.000,1.750,0.974502,1
2,Paris St-G,St Etienne,2025-01-12,France,Ligue 1,17,1,1,15,17,...,75.000,62.500,3.000,0.000,0.000,75.000,62.500,0.625,0.770152,1
3,Dortmund,Leverkusen,2025-01-10,Germany,Bundesliga,16,6,1,2,3,...,62.500,37.500,2.500,57.143,28.571,57.143,42.857,2.143,0.655196,1
4,St Pauli,Eintracht Frankfurt,2025-01-11,Germany,Bundesliga,16,14,16,3,6,...,28.571,14.286,0.429,50.000,25.000,75.000,37.500,2.000,0.632550,1
5,Hoffenheim,Wolfsburg,2025-01-11,Germany,Bundesliga,16,15,12,11,5,...,75.000,62.500,2.000,57.143,57.143,71.429,71.429,2.571,0.719829,1
6,Mgladbach,Bayern Munich,2025-01-11,Germany,Bundesliga,16,8,4,1,1,...,62.500,62.500,2.250,62.500,37.500,75.000,62.500,3.125,0.627316,1
7,Mainz,Bochum,2025-01-11,Germany,Bundesliga,16,5,11,18,18,...,37.500,12.500,1.250,0.000,0.000,50.000,37.500,0.875,0.644169,1
8,Lazio,Como,2025-01-10,Italy,Serie A,20,4,4,16,17,...,88.889,33.333,2.000,10.000,0.000,40.000,30.000,0.800,0.976199,1
9,Empoli,Lecce,2025-01-11,Italy,Serie A,20,12,18,18,19,...,22.222,0.000,0.333,11.111,0.000,22.222,11.111,0.222,1.000000,1


In [30]:
fl.create_import_file(all_predictions, r"C:\Users\leere\OneDrive\Desktop\IMPORTS\LAY_U25_TOP_5_IMPORT.csv", provider="lay_u25", market_name="Over/Under 2.5 Goals", selection_name="Under 2.5 Goals")

File created and saved successfully at: C:\Users\leere\OneDrive\Desktop\IMPORTS\LAY_U25_TOP_5_IMPORT.csv
